In [1]:
from aicspylibczi import CziFile
import numpy as np
import skimage as sk
import pandas as pd
import os
from glob import glob
import napari
import pyclesperanto as cle


In [ ]:
def normalize_images(input_image,tophat_radius):
    input_gpu = cle.push(input_image)
    #normalizing the image stack
    equalized_intensities_stack = cle.create_like(input_gpu)
    a_slice = cle.create([input_gpu.shape[-2], input_gpu.shape[-1]])
    num_slices = input_gpu.shape[1]
    num_channels = input_gpu.shape[0]
    if num_channels == 1:
        mean_intensity_stack = cle.mean_of_all_pixels(input_gpu)
        corrected_slice = None
        for z in range(0, num_slices):
            # get a single slice out of the stack
            cle.copy_slice(input_gpu, a_slice, z)
            # measure its intensity
            mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
            # correct the intensity
            correction_factor = mean_intensity_slice/mean_intensity_stack
            corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
            # copy slice back in a stack
            cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    else:
        for channel in range(num_channels):
            mean_intensity_stack = cle.mean_of_all_pixels(input_gpu[channel,...])
            corrected_slice = None
            for z in range(0, num_slices):
                # get a single slice out of the stack
                cle.copy_slice(input_gpu[channel,...], a_slice, z)
                # measure its intensity
                mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
                # correct the intensity
                correction_factor = mean_intensity_slice/mean_intensity_stack
                corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
                # copy slice back in a stack
                cle.copy_slice(corrected_slice, equalized_intensities_stack[channel], z)
    #background subtraction (increase the signal to noise ratio for improved segmentation results)
    background_subtracted_top_hat = cle.top_hat_sphere(equalized_intensities_stack,radius_x=tophat_radius,radius_y=tophat_radius,radius_z=tophat_radius)
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    background_subtracted_top_hat_pull = cle.pull(background_subtracted_top_hat)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    return background_subtracted_top_hat_pull

In [5]:
file_paths = sorted(glob("E:\Fondufe-Mittendorf_Lab\OIC-150_Images\*.czi"))
czi_files = list(map(CziFile,file_paths))

In [11]:
test_img = czi_files[0]

In [12]:
test_img.dims

'BVSTCZYX'

In [13]:
test_img.size

(1, 1, 3, 1, 4, 24, 1012, 1012)

In [21]:
help(CziFile.read_image)

Help on function read_image in module aicspylibczi.CziFile:

read_image(self, **kwargs)
    Read the subblocks in the CZI file and for any subblocks that match all the constraints in kwargs return
    that data. This allows you to select channels/scenes/time-points/Z-slices etc. Note if passed a BGR image
    then the dims of the object will returned by this function and the implicit BGR will be expanded into an
    A dimension. A is samples per pixel and will only be present for BGR images. This is logically more consistent
    than mixing the samples into the channels as was done before aicspylibczi-3.0.0.
    
    Parameters
    ----------
    **kwargs
        The keywords below allow you to specify the dimensions that you wish to match. If you
        under-specify the constraints you can easily end up with a massive image stack.
             Z = 1   # The Z-dimension.
             C = 2   # The C-dimension ("channel").
             T = 3   # The T-dimension ("time").
             

In [ ]:
img = [test_img.read_image(S=i)[0] for i in range(test_img.size[2])]

In [28]:
img = np.squeeze(img)

In [29]:
img[0].shape

(4, 24, 1012, 1012)